# Applying score matching to real data
Using data from /nfs/nhome/live/maneesh/Work/motor-variability/Decoding1ms/MotorHam1.mat

Maneesh comments:
The matrix S contains spikes sampled at 1ms: Neuron x TimeBin x Trial.
Trials are shorter than size(S,3) -- so maybe restrict to first 1100
ms or so.  This is all one reach direction -- other MotorHam? files
have other reach directions.


In [145]:
using MAT # to load mat file
using NBInclude

In [146]:
# Load some data
file = matopen("MotorHam1.mat")
rawdata = read(file, "S")
size(rawdata) #neuron by timebin by trial

(199,1500,216)

In [149]:
# read the data into our input format, trial x (neuron) x ((spiketimes)))
hasfired = zeros(size(rawdata,1),1) # track if a neuron has fired ever
data = Array(Any, (size(rawdata,3),))
for n = 1:length(data)
    data[n] = Array(Any, (size(rawdata,1),))
    for m = 1:size(rawdata,1)        
        data[n][m] = find(x->(x>=1), rawdata[m,201:1200,n])*1e-3 # set of spiketimes between 0 and 1 sec
        if length(data[n][m]) > 0
            hasfired[m] = 1
        end
    end
end

# trim the data of neurons that are silent on every trial (no spikes whatsoever within the whole dataset)
for n = 1:length(data)
    data[n] = data[n][find(x->x==1, hasfired)]
end
        

In [182]:
# Load the processing files
nbinclude("ScoreMatching_functions.ipynb")

coordinate_descent (generic function with 1 method)

In [154]:
# Setup score matching, ex_dims = [M, D, F, N]. 
ex_dims = [5, 2, 8, 3]
rand_params = create_rand_params(ex_dims...); # Assumes T=1 (true) and ω=1 (true)
rand_params.T = 1000*1e-3
rand_params.ω = 1./rand_params.T
rand_params.C = 0.1*rand_params.C
rand_params.θ = 0.1*rand_params.θ
;

In [183]:
# Run coordinate descent

par = getall(rand_params);

penalise_integrals = Array(Any, 3)
penalise_integrals[2] = 1e2;
penalise_integrals[3] = [];

par, S = coordinate_descent(J, data, getall(rand_params), max_iter=3, verbose=1, penalise_integrals=penalise_integrals);
#par, S = coordinate_descent(J, data, par, max_iter=3, verbose=1, penalise_integrals=0);

(N,M,D,F) = (3,5,2,8)
Initial cost is 37177.600691379295
After setting b, cost is 513.8526494656347


Starting iteration 1...............
Updated θ ----------------- Current cost is -428.96837865508735
Updated C ----------------- Current cost is -6682.757367240293
Updated ϕ_1 ------------ Current cost is -7064.330694792394
Updated ϕ_2 ------------ Current cost is -9645.060919392763
Updated ϕ_3 ------------ Current cost is -12458.433746490062
Updated suff stats --------- Current cost is -12458.433746490064


Starting iteration 2...............
Updated θ ----------------- Current cost is -28934.538944131917
Updated C ----------------- Current cost is -30089.60200685986
Updated ϕ_1 ------------ Current cost is -31487.216017847903
Updated ϕ_2 ------------ Current cost is -31753.119430738134
Updated ϕ_3 ------------ Current cost is -32478.06935437386
Updated suff stats --------- Current cost is -32478.069354373867


Starting iteration 3...............
Updated θ ----------------- Current cos

In [185]:
using JLD
save("MotorHam1_results_new_weaker_constraint.jld", "par", par, "S", S, "data", data, "rand_params", rand_params, "penalise_integrals", penalise_integrals)

In [190]:
# Plot the results
# Plot data, true rate function and estimated rate function for a neuron on given trials
using PlotlyJS

num_neur = 1
trials = 1:3
dim_latent = 2

#Plot estimated
to_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
rate_rand_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
rate_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
latent_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
latent_est_plot = Array(PlotlyJS.GenericTrace{Dict{Symbol,Any}}, size(data,1))
colors = ["rgb(0,0,255)", "rgb(0,255,0)", "rgb(255,0,0)", "rgb(128,128,0)", "rgb(0,128,128)"]
for i1 = trials
    @show collect(data[i1][num_neur])
    # Plot data
    to_plot[i1] = scatter(;x=collect(data[i1][num_neur]), y=i1*collect(ones(size(data[i1][num_neur]))), mode="markers", marker_color=colors[mod(i1,5)+1], yaxis="y2")
    
    # Compute underlying rate and latent functions
    rate_est = lambda_functions(0:0.01:rand_params.T, par[1], par[2]; b=par[4], phi=par[3][:,i1], omega=par[5])
    rate_rand = lambda_functions(0:0.01:rand_params.T, rand_params.C, rand_params.θ; b=rand_params.b, phi=rand_params.ϕ[:,i1], omega=rand_params.ω)
    latent_est = latent_functions(0:0.01:rand_params.T, par[2]; phi=par[3][:,i1], omega=par[5])
    
    # Plot underlying functions
    rate_est_plot[i1] = scatter(; 
    x=collect(0:0.01:rand_params.T), 
    y=collect(rate_est[num_neur,:]),
    line_color = colors[mod(i1,5)+1],
    yaxis = "y0"
)

    rate_rand_plot[i1] = scatter(; 
    x=collect(0:0.01:rand_params.T), 
    y=collect(rate_rand[num_neur,:]),
    line_color = colors[mod(i1,5)+1],
    yaxis = "y0"
)

    latent_est_plot[i1] = scatter(; 
    x=collect(0:0.01:rand_params.T), 
    y=collect(latent_est[dim_latent,:]),
    line_color = colors[mod(i1,5)+1],
    yaxis = "y0"
)

end

lo = Layout(;xaxis_range=[0,rand_params.T])

#plt = plot([Plot(to_plot[trials]); Plot(rate_plot[trials]);  Plot(rate_est_plot[trials]);  Plot(latent_plot[trials]);  Plot(latent_est_plot[trials])])
plt = plot([Plot(to_plot[trials], lo); Plot(rate_est_plot[trials], lo); Plot(rate_rand_plot[trials],lo)])

relayout!(plt, height=700)

plt

collect((data[i1])[num_neur]) = [0.641,0.662,0.664,0.725,0.752,0.754,0.795]
collect((data[i1])[num_neur]) = [0.588,0.621,0.626,0.653,0.714,0.716,0.766]
collect((data[i1])[num_neur]) = [0.591,0.639,0.788]


In [ ]:
par[4]

In [191]:
# Show number of spikes, integral of true rate function and integral of estimated rate function
for j1 = 1
for i1=1:ex_dims[4]
        bla_est = lambda_functions(0:0.01:rand_params.T, par[1], par[2]; b=par[4], phi=par[3][:,i1], omega=par[5])

        a= [size(data[i1][j1],1)/rand_params.T (sum(bla_est[j1,:])/length(bla_est[j1,:]))]
    @show a
    end
end

a = [7.0 8.329883337160345]
a = [7.0 8.289868654043977]
a = [3.0 8.32854720322015]


In [123]:
par[1]

30x3 Array{Float64,2}:
   -0.6915         2.44339         0.107187  
    0.302269      -0.519222        1.91846   
   -2.05973       -4.24297        -0.539133  
   -2.06977       -1.48496         0.677428  
    0.913563      -0.968573       -1.73208   
   -1.51529        0.809484       -1.31412   
    2.396         -2.02391        -1.90115   
   -2.3208         0.431386       -0.760968  
   -3.94734        0.752973       -2.32966   
 9019.16        7226.17        13181.2       
    2.86957e14    -5.70346e15     -1.27214e15
    2.92887       -2.47504         6.27024   
   -0.526414      -4.84765        -4.07618   
    ⋮                                        
    0.489442      -0.167923       -1.2774    
   12.9512        -4.44786        12.2872    
   -0.776951      -1.88176         1.53486   
    0.885406      -1.1339         -0.359844  
   -2.51199       -3.39978         0.332203  
    0.323933      -3.99719         1.19433   
    4.43369        1.99933         3.27042   
   -0.96139

In [66]:
exp(-1)

0.36787944117144233

In [178]:
num_neur = 4
[((par[1]*par[2])[num_neur,:]').^2 rand_params.ω*(1:ex_dims[3])] # How much power from each frequency gets mapped from to the neuron

8x2 Array{Float64,2}:
 0.0015009    1.0
 2.91483e-5   2.0
 1.71777e-5   3.0
 6.33382e-6   4.0
 0.000657702  5.0
 0.00100582   6.0
 0.00407983   7.0
 0.0071517    8.0